AOC [23.03](https://adventofcode.com/2023/day/3)

Puzzle 1:
Goldola isn't working b/c engine is missing a part.
Engine schematic with numbers and symbols (*,#,+). Any number adjacent (vert, horz, diag) to symbol is a part number. Find all such numbers and add together.

How to solve:
* tricky b/c they want the whole number, not just the digit that is adjacent to the symbol.
* is the solution to the exmaple a hint? assume numbers to be summed unless they are not adjacent? If so, find all numbers and then subtract those that are not adjacent?

1. Easy to extract the numbers with a RegEx; this will also report where the number apperars in the line. So I could create a list / dict of numbers and their row/col values.
1. Easy to extract the symbols, too; and create a list / dict of their row/col values.
1. Then figure out some way of looking for overlaps in the lists of row/col values?

Let's get started on the extraction / list bits and perhaps that will help clarify the testing method.

In [6]:
sampleData = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''

import re
findNumbers  = re.compile(r'\d+')
findSymbols = re.compile(r'[\x21-\x2D]|[\x2F]')
                         
linenum = 0
for line in sampleData.split('\n'):
    nums = re.findall(findNumbers,line)
    symbs = re.search(findSymbols,line)
    print(f"line number {linenum}")
    print(f"numbers found {nums}")
    print(f"symbols found {symbs}")
    linenum += 1




line number 0
numbers found ['467', '114']
symbols found None
line number 1
numbers found []
symbols found <re.Match object; span=(3, 4), match='*'>
line number 2
numbers found ['35', '633']
symbols found None
line number 3
numbers found []
symbols found <re.Match object; span=(6, 7), match='#'>
line number 4
numbers found ['617']
symbols found <re.Match object; span=(3, 4), match='*'>
line number 5
numbers found ['58']
symbols found <re.Match object; span=(5, 6), match='+'>
line number 6
numbers found ['592']
symbols found None
line number 7
numbers found ['755']
symbols found None
line number 8
numbers found []
symbols found <re.Match object; span=(3, 4), match='$'>
line number 9
numbers found ['664', '598']
symbols found None


Annoying: 
* search() returns both the matched item and where (span) it ocurrs, but only for the first value
* findall() returns all ocurrances of the matchs but without location information

Let's try the finditer() method to see if we can get both pieces of information

In [18]:
linenum = 0
for line in sampleData.split('\n'):
    for match in re.finditer(findNumbers,line):
        print(linenum,match.span(),match.group())
    for match in re.finditer(findSymbols,line):
        print(linenum,match.span(),match.group())
    linenum += 1
    

0 (0, 3) 467
0 (5, 8) 114
1 (3, 4) *
2 (2, 4) 35
2 (6, 9) 633
3 (6, 7) #
4 (0, 3) 617
4 (3, 4) *
5 (7, 9) 58
5 (5, 6) +
6 (2, 5) 592
7 (6, 9) 755
8 (3, 4) $
8 (5, 6) *
9 (1, 4) 664
9 (5, 8) 598


In [57]:
def adjacent(point1,point2):
    testX = [point1[0]+i for i in range(-1,2)]
    testY = [point1[1]+i for i in range(-1,2)]
    #print(f"testX:{testX}, testY{testY}")
    if (point2[0] in testX) and (point2[1] in testY):
        return True
    else:
        return False

In [54]:
numberList = []
symbolList = []
linenum = 0
for line in sampleData.split('\n'):
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
        #symbolList.append(points)
    linenum += 1    

numberList
#symbolList

[['467', [(0, 0), (1, 0), (2, 0)]],
 ['114', [(5, 0), (6, 0), (7, 0)]],
 ['35', [(2, 2), (3, 2)]],
 ['633', [(6, 2), (7, 2), (8, 2)]],
 ['617', [(0, 4), (1, 4), (2, 4)]],
 ['58', [(7, 5), (8, 5)]],
 ['592', [(2, 6), (3, 6), (4, 6)]],
 ['755', [(6, 7), (7, 7), (8, 7)]],
 ['664', [(1, 9), (2, 9), (3, 9)]],
 ['598', [(5, 9), (6, 9), (7, 9)]]]

In [55]:
symbolList

[(3, 1), (6, 3), (3, 4), (5, 5), (3, 8), (5, 8)]

In [59]:
for number in numberList:
    partNumber = number[0]
    points = number[1]
    for point in points: # look at each point of a number list
        for symbolpoint in symbolList:
            if adjacent(point,symbolpoint):
                print(partNumber)
                break

467
35
35
633
633
617
592
755
664
664
598
598


Some numbers are doubled (35, 633) when two of their digits are adjacent to a symbol square. Let's move the test into a procedure that we can return out of as soon as we get one positive hit.

In [60]:
def testNumber(points,symbolList):
    for point in points: # look at each point of a number list
        for symbolpoint in symbolList:
            if adjacent(point,symbolpoint):
                return True
    return False

In [62]:
for number in numberList:
    partNumber = number[0]
    points = number[1]
    if testNumber(points,symbolList):
        print(partNumber)

467
35
633
617
592
755
664
598


OK, that works. Pull it all together:

In [66]:
def adjacent(point1,point2):
    testX = [point1[0]+i for i in range(-1,2)]
    testY = [point1[1]+i for i in range(-1,2)]
    #print(f"testX:{testX}, testY{testY}")
    if (point2[0] in testX) and (point2[1] in testY):
        return True
    else:
        return False

def testNumber(points,symbolList):
    for point in points: # look at each point of a number list
        for symbolpoint in symbolList:
            if adjacent(point,symbolpoint):
                return True
    return False

import re
findNumbers  = re.compile(r'\d+')
findSymbols = re.compile(r'[\x21-\x2D]|[\x2F]')

numberList = []
symbolList = []
linenum = 0
for line in sampleData.split('\n'):
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
        #symbolList.append(points)
    linenum += 1 

tally = 0    
for number in numberList:
    partNumber = number[0]
    points = number[1]
    if testNumber(points,symbolList):
        tally += int(partNumber)

print(f"part number tally is {tally}")

part number tally is 4361


In [69]:
# test code works; get the real data...

with open ('2303input.txt') as f_in:
    data = [line for line in f_in.read().split('\n')]

numberList = []
symbolList = []
linenum = 0
for line in data:
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
        #symbolList.append(points)
    linenum += 1 

tally = 0    
for number in numberList:
    partNumber = number[0]
    points = number[1]
    if testNumber(points,symbolList):
        tally += int(partNumber)

print(f"part number tally is {tally}")

part number tally is 490593


This is too low. Perhaps I don't have all the symbols? Let's find the symbols and change regex expresion to look for just THOSE.

In [77]:
chars = []
findNumbers  = re.compile(r'[^\d]')
for line in data:
    for match in re.finditer(findNumbers,line):
        chars.append(match.group())

print(set(chars))


{'.', '=', '-', '/', '%', '#', '&', '$', '+', '@', '*'}


In [78]:
import re
findNumbers  = re.compile(r'\d+')
findSymbols = re.compile(r'[=\-/%#&$+@*]')

with open ('2303input.txt') as f_in:
    data = [line for line in f_in.read().split('\n')]

numberList = []
symbolList = []
linenum = 0
for line in data:
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
        #symbolList.append(points)
    linenum += 1 

tally = 0    
for number in numberList:
    partNumber = number[0]
    points = number[1]
    if testNumber(points,symbolList):
        tally += int(partNumber)

print(f"part number tally is {tally}")


part number tally is 533775


That's it!

Part 2:
look for astrick (*) that is adjacent to two numbers.
multiply the two numbers.
tally the products.

Solve:
1. Similar procedure, but this time just find the stars' locations
1. loop through the stars (not numbers)
1. for each star, see if it's adjacent to two numbers and if so, do the math.

In [79]:
import re
findNumbers  = re.compile(r'\d+')
findSymbols = re.compile(r'[*]')

numberList = []
symbolList = []
linenum = 0
for line in sampleData.split('\n'):
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
        #symbolList.append(points)
    linenum += 1 

symbolList

[(3, 1), (3, 4), (5, 8)]

In [80]:
numberList

[['467', [(0, 0), (1, 0), (2, 0)]],
 ['114', [(5, 0), (6, 0), (7, 0)]],
 ['35', [(2, 2), (3, 2)]],
 ['633', [(6, 2), (7, 2), (8, 2)]],
 ['617', [(0, 4), (1, 4), (2, 4)]],
 ['58', [(7, 5), (8, 5)]],
 ['592', [(2, 6), (3, 6), (4, 6)]],
 ['755', [(6, 7), (7, 7), (8, 7)]],
 ['664', [(1, 9), (2, 9), (3, 9)]],
 ['598', [(5, 9), (6, 9), (7, 9)]]]

In [92]:
for gear in symbolList: # each is a tuple (x,y)
    pairs=[]
    for number in numberList:
        points = number[1]
        for point in points:
            if adjacent(gear,point): 
                pairs.append(int(number[0]))
                print(pairs)
            if len(pairs) == 2:
                print(f"pairs {pairs},product = {pairs[0] * pairs[1]}")
                break


[467]
[467, 35]
pairs [467, 35],product = 16345
pairs [467, 35],product = 16345
pairs [467, 35],product = 16345
pairs [467, 35],product = 16345
pairs [467, 35],product = 16345
pairs [467, 35],product = 16345
pairs [467, 35],product = 16345
pairs [467, 35],product = 16345
[617]
[755]
[755, 598]
pairs [755, 598],product = 451490


Again, this works but I need to put it into a function so we can return as soon as we find two numbers adjacent to a gear.

In [107]:
def adjacent(point1,point2):
    testX = [point1[0]+i for i in range(-1,2)]
    testY = [point1[1]+i for i in range(-1,2)]
    #print(f"testX:{testX}, testY{testY}")
    if (point2[0] in testX) and (point2[1] in testY):
        return True
    else:
        return False

def findPairProduct(gear,numberList):
    pairs = []
    for number in numberList:
        points = number[1]
        for point in points:
            if adjacent(gear,point): 
                pairs.append(int(number[0]))
                print(pairs)
            if len(pairs) == 2:
                return int(pairs[0]) * int(pairs[1])
    return 0

import re
findNumbers  = re.compile(r'\d+')
findSymbols = re.compile(r'[*]')

numberList = []
symbolList = []
linenum = 0
for line in sampleData.split('\n'):
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
        #symbolList.append(points)
    linenum += 1 

tally = 0    
for gear in symbolList:
    tally += findPairProduct(gear,numberList)

print(f"part number tally is {tally}")

[467]
[467, 35]
[617]
[755]
[755, 598]
part number tally is 467835


In [109]:
with open ('2303input.txt') as f_in:
    data = [line for line in f_in.read().split('\n')]

numberList = []
symbolList = []
linenum = 0
for line in data:
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
        #symbolList.append(points)
    linenum += 1 

tally = 0    
for gear in symbolList:
    tally += findPairProduct(gear,numberList)

print(f"part number tally is {tally:,}")

[702]
[785]
[785, 14]
[284]
[284, 453]
[503]
[503, 503]
[995]
[995, 995]
[458]
[458, 458]
[287]
[822]
[822, 822]
[174]
[174, 511]
[711]
[711, 320]
[634]
[634, 634]
[223]
[223, 933]
[522]
[522, 522]
[476]
[476, 476]
[424]
[424, 618]
[338]
[338, 363]
[205]
[205, 960]
[676]
[676, 172]
[763]
[996]
[996, 911]
[996]
[996, 996]
[566]
[566, 890]
[802]
[802, 93]
[214]
[214, 209]
[527]
[527, 527]
[928]
[928, 928]
[741]
[741, 741]
[18]
[18, 18]
[624]
[624, 624]
[64]
[64, 266]
[701]
[701, 458]
[721]
[721, 721]
[502]
[502, 80]
[25]
[25, 25]
[831]
[831, 831]
[377]
[377, 266]
[236]
[236, 631]
[938]
[938, 938]
[973]
[973, 341]
[738]
[738, 231]
[428]
[428, 428]
[311]
[311, 775]
[48]
[48, 19]
[258]
[258, 258]
[257]
[257, 459]
[807]
[807, 807]
[906]
[906, 906]
[185]
[185, 503]
[778]
[778, 778]
[251]
[251, 251]
[702]
[702, 702]
[941]
[941, 941]
[710]
[710, 733]
[685]
[685, 685]
[171]
[171, 508]
[658]
[658, 617]
[170]
[170, 170]
[874]
[874, 989]
[208]
[210]
[210, 210]
[129]
[129, 129]
[931]
[931, 618]
[979

101019153 is incorrect; it is too large

The problem was that the test was reporting numbers twice if the star was adjacent to two digits.

Fixed this by using a "break" to get out of the number loop as soon as ANY digit was found to be adjacent.

In [106]:
def adjacent(point1,point2):
    testX = [point1[0]+i for i in range(-1,2)]
    testY = [point1[1]+i for i in range(-1,2)]
    #print(f"testX:{testX}, testY{testY}")
    if (point2[0] in testX) and (point2[1] in testY):
        return True
    else:
        return False

def findPairProduct(gear,numberList): # tuple,[number,[tuple,tuple,tuple]] - list of x,y coords for the digits in the number
    pairs = []
    for number in numberList: 
        points = number[1] # get the list of tuples for digit locations
        for point in points:
            if adjacent(gear,point): 
                pairs.append(int(number[0]))
                #print(pairs)
                break # as soon as we get a match for a number, go to the next number in the list
        if len(pairs) == 2: # as soon as we get a pair of numbers, return the product
            return int(pairs[0]) * int(pairs[1])
    return 0

with open ('2303input.txt') as f_in:
    data = [line for line in f_in.read().split('\n')]

import re
findNumbers  = re.compile(r'\d+')
findSymbols = re.compile(r'[*]')

numberList = []
symbolList = []
linenum = 0
for line in data:
    for match in re.finditer(findNumbers,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            points.append((i,linenum))
        numberList.append([match.group(),points]) # create list of numbers and locations

    for match in re.finditer(findSymbols,line):
        points = []
        for i in range(match.span()[0],match.span()[1]): 
            symbolList.append((i,linenum))
    linenum += 1 

tally = 0    
for gear in symbolList:
    tally += findPairProduct(gear,numberList)

print(f"part number tally is {tally}")

part number tally is 78236071
